## High-level overview

We can all agree researching products, platforms or services is painful and time taking.

I often find myself researching multiple sources (R.I.P chrome tabs) to see if something is reliable, worthy, and reasonable to buy and still feel like I am lost. Information is scattered, hard to find, and hard to sort through (multiple mentions of different products, mixed reviews) and get a clear summary.

Moreover, Misleading ads, SEO spam (best 5 product articles with the least research), and fake reviews (Affiliated links, sponsored content) can make the whole experience frustrating.

Reddit is one place where people leave honest reviews and discourage any fake ones. With over 52 million daily active users, Reddit provides a platform for people from all walks of life to share their experiences and opinions.

### The goal of this project is to help users find:
1) if a product or service is worth it via sentiment analysis. This helps with queries such as "Is regal unlimited subscription worth it?"
2) product mentions amongst posts and comments via named entity recognition (NER). This helps with queries such as "Best 4K TV to buy".

Please see the set_up_notebook.ipynb that addresses first problem - sentiment analysis.

### This notebook focuses on solving the second problem - Named Entity Recognition

Hugging Face's Transformers provides State-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX.
Training an NLP model from scratch takes hundreds of hours.
Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce compute costs, carbon footprint, and save the time and resources required to train a model from scratch.


https://huggingface.co/models

Although we can get a pre-trained model for our task (NER), these pre-trained models are not trained on the product entities. We need to get a dataset that has product labels and fine tune the pre-trained model on product enntities to be able to predict product mentions from reddit.

Let's load libraries!

In [23]:
# Loading libraries

import os
from datasets import load_dataset, concatenate_datasets, load_metric
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, create_optimizer, AutoModel
from transformers import EarlyStoppingCallback
import evaluate
import numpy as np
import pandas as pd
#import tensorflow as tf
#from transformers.keras_callbacks import KerasMetricCallback
#from transformers.keras_callbacks import PushToHubCallback
from transformers import pipeline
import wandb


Let's get the dataset that has product labels from HugginFace Datasets library.

In [2]:
wnut = load_dataset("wnut_17")
wnut

Reusing dataset wnut_17 (C:\Users\avanjavakam\.cache\huggingface\datasets\wnut_17\wnut_17\1.0.0\077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

## Description of Input Data

WNUT 17: Emerging and Rare entity recognition (https://huggingface.co/datasets/wnut_17)
This dataset was chosen among many available for the following reasons:
1) Reddit has many previously-unseen entities in the context of emerging discussions which is what this dataset focuses on
2) Size of the dataset is small but good enough to meet the project goals
3) Faster training (relative to other large datasets) and GPU is not required
4) It is trained on noisy user-generated text (similar to what's needed for this project)

train dataset has 3394 rows, validation dataset has 1009 rows and test dataset has 1287 rows.

Let us see pre-defined entities of the data. Product entities are of the most interest for this project.

In [47]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
id2tag = {id: tag for id, tag in enumerate(label_list)}
id2tag

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

In [46]:
label_list = wnut["test"].features[f"ner_tags"].feature.names
id2tag = {id: tag for id, tag in enumerate(label_list)}
id2tag

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

Train data set is not large but transformers require a large data set.
To address this, train and validation sets are combined. Test data set will be untouched for evaluation.

In [4]:
# merge train & validation sets
train_dataset = concatenate_datasets([wnut["train"],wnut["validation"]])
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 4403
})

In [5]:
train_dataset[2]

{'id': '2',
 'tokens': ['Pxleyes',
  'Top',
  '50',
  'Photography',
  'Contest',
  'Pictures',
  'of',
  'August',
  '2010',
  '...',
  'http://bit.ly/bgCyZ0',
  '#photography'],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [6]:
# looking at an example
ith_example=2

print(wnut["train"][ith_example]['tokens'])
print([id2tag[label] for label in train_dataset[ith_example]['ner_tags']])

['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']
['B-corporation', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


Pxleyes is tagged as a corporation and the rest are 'O' which means they do not belong to any entity

## Strategy for solving the problem

1) First, a pre-trained model is needed from the Hugging Face.
    Encoder	models such as ALBERT, BERT, DistilBERT, ELECTRA, RoBERTa can be used for NER (also called token-classification).

    These models are really large. I chose DistilBERT for it's faster training while meeting the functionality.

    Distilbert aims to optimize the training by reducing the size of BERT and increase the speed of BERT — all while trying to retain as much performance as possible. Specifically, Distilbert is 40% smaller than the original BERT-base model, is 60% faster than it, and retains 97% of its functionality.
2) This model needs to be fine tuned with wnut dataset in order to predict product entities.
3) Evaluate how the model performs with test dataset based on accuracy (why this metric was chosen will be explained below)
4) Test with a few example comments from reddit
5) Push the model to Hugging Face hub so it can be used by the streamlit app for inference

## Discussion of the expected solution

For example, a user searches "Best 4K TV to buy" on the app, the model needs to identify the products that are mentioned in all the comments and posts on Reddit.

For example,
text="""Sony X90K is one of the best TVs ever if you are in a room that has a lot of sunlight. But if you game a lot, nothing can beat LG C2 OLED. If you are tight on budget, go with TCL 6 series"""

The model should tag Sony X90K, LG C2 OLED and TCL 6 Series as products.

## Metrics with justification

Accuracy can be priortized over F1 score for this classification task considering the context and consequences.

1) There are no siginificant consequences for false positives or false negatives.

2) F1 score improvement requires a very balanced dataset (no of entities and non-entities being roughly equal) which is not the case for this project where text will have extremely high non-entities and only a handful of entities. F1 score is influenced by recall, which can be significantly impacted by imbalanced datasets.

3) Accuracy is label-level metric that measures the overall correctness of the predicted labels compared to the true labels. It provides a straightforward and intuitive measure of how well the model predicts the entities as a whole. As the primary focus of this NER task is on overall correct labeling, accuracy is a suitable metric.

4) It is hard to find a good decision threshold (usually 0.5) to provide a better trade-off between precision and recall for this specific problem as an entity and a non-entity should both be predicted correctly.

5) Optimizing for F1 score requires annotated reddit dataset (fine tuning specifically for products) which I couldn't find and may involve manual labelling which is extremely expensive in terms of time and cost.

6)  Accurately recognizing non-entities (tokens that do not correspond to any entity) is equally important as identifying the correct entity spans. Accuracy considers both entity and non-entity predictions, providing an overall measure of the model's performance in recognizing both types



## EDA

Get the pre-trained distilbert model and it's tokenizer.

In [7]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

exploring how tokenizer behaves with an example

In [8]:
tokenized_input = tokenizer(wnut["train"][2]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

This is how the original input looks

In [9]:
#input
print(wnut["train"][2]["tokens"])

['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']


Distilbert's tokenizer splits them into sub tokens. Special tokens CLS and SEP are added too. This results in a mismatch between the inputs and the labels.

In [10]:
#tokenized
print(tokens)

['[CLS]', 'p', '##xley', '##es', 'top', '50', 'photography', 'contest', 'pictures', 'of', 'august', '2010', '.', '.', '.', 'http', ':', '/', '/', 'bit', '.', 'l', '##y', '/', 'b', '##gc', '##y', '##z', '##0', '#', 'photography', '[SEP]']


## Data Preprocessing

Re-alignment of tokens and labels involves:
1. Each token is mapped to its tag
2. assign label -100 to unnecessary tokens. PyTorch ignores -100 value during loss calculation.
3. Sub tokens such as [‘p’, ‘##xley’, ‘##es’] should become [1,-100,-100] so we can re align.



In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
# applying the custom token function
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)

Parameter 'function'=<function tokenize_and_align_labels at 0x000002488D9641F0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [13]:
id2tag[-100]='ignore'
exml=tokenized_train_dataset[2]

pd.DataFrame({'tokens':tokenizer.convert_ids_to_tokens(exml["input_ids"]), 'ner_labels':exml['labels'], 'ner_tags': [id2tag[label] for label in exml['labels']] })

,tokens,ner_labels,ner_tags
0,[CLS],-100,ignore
1,p,1,B-corporation
2,##xley,-100,ignore
3,##es,-100,ignore
4,top,0,O
5,50,0,O
6,photography,0,O
7,contest,0,O
8,pictures,0,O
9,of,0,O


## Modeling

Initial check:
Check how a simple baseline model performs. This model can just tag every token with the most frequent entity throughout the data which is O.

This resulted in ~59% accuracy.

In [22]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(pd.Series(tokenized_train_dataset['input_ids']).explode(), pd.Series(tokenized_train_dataset['labels']).explode().astype(str))
dummy_clf.score(pd.Series(tokenized_train_dataset['input_ids']).explode(), pd.Series(tokenized_train_dataset['labels']).explode().astype(str))

0.5888494815191806

The baseline classifier becomes less naive if we tag each token with the most frequent label of the sentence it belongs.
This resulted in 72% accuracy.

In [23]:
exploded_values=pd.Series(tokenized_train_dataset['labels']).explode()
exploded_values=pd.DataFrame(exploded_values,columns=['B'])

most_frequent_elem_by_doc=pd.Series(tokenized_train_dataset['labels']).apply(lambda x:  max(set(x), key=x.count))
most_frequent_elem_by_doc=pd.DataFrame(most_frequent_elem_by_doc,columns=list('A'))

df_most_freq_token=exploded_values.merge(most_frequent_elem_by_doc, how='right', left_index=True, right_index=True)

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(pd.Series(tokenized_train_dataset['input_ids']).explode(), df_most_freq_token['A'])
dummy_clf.score(pd.Series(tokenized_train_dataset['input_ids']).explode(), df_most_freq_token['A'])

0.7197897448947134

### Using DistilBERT for Named Entity Recognition

labels should be padded the exact same way as the inputs so that they stay the same size, using -100 as a value so that the corresponding predictions are ignored in the loss computation.

DataCollatorForTokenClassification helps with this padding.

In [14]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Get model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

a quick evaluation before we fine tune the model

In [27]:
metric_seqeval = load_metric("seqeval")
example = wnut["train"][2]

labels = [label_list[i] for i in example["ner_tags"]]
metric_seqeval.compute(predictions=[labels], references=[labels])

{'corporation': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

We get the precision, recall, and F1 score for each separate entity, as well as overall.
To get metrics on the validation set during training, we need to define the function that'll calculate the metric for us. This is very well-documented in their official docs.

In [39]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric_seqeval.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
    # adding entity level metric as well
    #for k in results.keys():
    #    if k not in flattened_results.keys():
    #        flattened_results[k+"_f1"]=results[k]["f1"]
    return flattened_results

In [59]:


training_args = TrainingArguments(
    output_dir='C:/Users/avanjavakam/OneDrive - Moulton Niguel Water/Documents/R_Home/distilbert_ner',
    #report_to="wandb",
    #run_name = "initial_run"
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500, 
    eval_steps=60,
    save_steps=60,
    evaluation_strategy="steps",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 6)]
)

trainer.train()
#wandb.finish()

c:\Users\avanjavakam\.conda\envs\lit_or_not_on_reddit\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1380 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/21 [00:00<?, ?it/s]

c:\Users\avanjavakam\.conda\envs\lit_or_not_on_reddit\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 0.26181262731552124, 'eval_precision': 0.5732814526588845, 'eval_recall': 0.40963855421686746, 'eval_f1': 0.47783783783783784, 'eval_accuracy': 0.9491257321191912, 'eval_runtime': 85.1954, 'eval_samples_per_second': 15.106, 'eval_steps_per_second': 0.246, 'epoch': 3.48}
{'loss': 0.0843, 'learning_rate': 8.636363636363637e-06, 'epoch': 3.62}


  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 0.2565685510635376, 'eval_precision': 0.5918097754293263, 'eval_recall': 0.4151992585727525, 'eval_f1': 0.48801742919389973, 'eval_accuracy': 0.9493394895472618, 'eval_runtime': 92.7674, 'eval_samples_per_second': 13.873, 'eval_steps_per_second': 0.226, 'epoch': 3.7}
{'train_runtime': 4502.8795, 'train_samples_per_second': 4.889, 'train_steps_per_second': 0.306, 'train_loss': 0.3002868703767365, 'epoch': 3.7}


TrainOutput(global_step=1020, training_loss=0.3002868703767365, metrics={'train_runtime': 4502.8795, 'train_samples_per_second': 4.889, 'train_steps_per_second': 0.306, 'train_loss': 0.3002868703767365, 'epoch': 3.7})

Test Set Evaluation

In [60]:
predictions, labels, _ = trainer.predict(tokenized_wnut["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric_seqeval.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/21 [00:00<?, ?it/s]

{'corporation': {'precision': 0.21008403361344538,
  'recall': 0.3787878787878788,
  'f1': 0.27027027027027023,
  'number': 66},
 'creative-work': {'precision': 0.3392857142857143,
  'recall': 0.13380281690140844,
  'f1': 0.1919191919191919,
  'number': 142},
 'group': {'precision': 0.3194444444444444,
  'recall': 0.1393939393939394,
  'f1': 0.19409282700421943,
  'number': 165},
 'location': {'precision': 0.49382716049382713,
  'recall': 0.5333333333333333,
  'f1': 0.5128205128205128,
  'number': 150},
 'person': {'precision': 0.5914972273567468,
  'recall': 0.745920745920746,
  'f1': 0.6597938144329898,
  'number': 429},
 'product': {'precision': 0.20481927710843373,
  'recall': 0.13385826771653545,
  'f1': 0.16190476190476194,
  'number': 127},
 'overall_precision': 0.46853823814133594,
 'overall_recall': 0.448563484708063,
 'overall_f1': 0.45833333333333337,
 'overall_accuracy': 0.9451498439570775}

Get Predictions

In [79]:
def tag_sentence(text:str):
    # convert our text to a  tokenized sequence
    inputs = tokenizer(text, truncation=True, return_tensors="pt")#.to("cuda")
    # get outputs
    outputs = model(**inputs)
    # convert to probabilities with softmax
    probs = outputs[0][0].softmax(1)
    # get the tags with the highest probability
    word_tags = [(tokenizer.decode(inputs['input_ids'][0][i].item()), id2tag[tagid.item()]) 
                  for i, tagid in enumerate (probs.argmax(axis=1))]

    return pd.DataFrame(word_tags, columns=['word', 'tag'])

few examples

In [62]:
text="""Apple unveils all-new MacBook Air, supercharged by the new M2 chip"""

print(tag_sentence(text))

         word            tag
0       [CLS]              O
1       apple  B-corporation
2          un              O
3        ##ve              O
4       ##ils              O
5         all              O
6           -              O
7         new              O
8         mac      B-product
9      ##book      I-product
10        air      I-product
11          ,              O
12      super              O
13  ##charged              O
14         by              O
15        the              O
16        new              O
17         m2      B-product
18       chip      I-product
19      [SEP]              O


In [1]:
from huggingface_hub import notebook_login

In [ ]:
!git config --global user.email "anudeepvanjavakam@gmail.com"
!git config --global user.name "anudeepvanjavakam1"


In [3]:
notebook_login()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [71]:
model = AutoModelForTokenClassification.from_pretrained("anudeepvanjavakam/distilbert_uncased_finetuned_wnut17")


In [72]:
trainer = Trainer(model=model)
tokenizer = AutoTokenizer.from_pretrained("anudeepvanjavakam/distilbert_uncased_finetuned_wnut17")

## Hyperparameter Tuning

In [16]:
#Set up Weights and Biases for tracking and monitoring model runs.
#Later we can use it for hyperparameter tuning.
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="reddit_product_tagging"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"


During hyperparameter search, the Trainer will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We just use the same function as before:

In [68]:
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

In [72]:
training_args = TrainingArguments(
    #output_dir='C:/Users/avanjavakam/OneDrive - Moulton Niguel Water/Documents/R_Home/distilbert_ner_wandb',
    report_to="wandb",
    run_name = "run_2_06_04_2023_11_44_pm",
    disable_tqdm=True,
    #num_train_epochs=5,
    #learning_rate=2e-5,
    #per_device_train_batch_size=16,   
    #per_device_eval_batch_size=64,
    #weight_decay=0.01,
    #warmup_steps=500, 
    eval_steps=500,
    #save_steps=60,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    
)

In [73]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

Population-Based Training
Population-based training uses guided hyperparameter search but does not need to restart training for new hyperparameter configurations. Instead of discarding bad-performing trials, we exploit good-performing runs by copying their network weights and hyperparameters and then explore new hyperparameter configurations while continuing to train.

The basic idea behind the algorithm in layman's terms:
Run the hyperparameter optimization process for some samples for a given time step (or iterations) T.

After every T iterations, compare the runs and copy the weights of good-performing runs to the bad-performing runs and change their hyperparameter values to be close to the runs' values that performed well.

Terminate the worst-performing runs.
Although the algorithm's idea seems simple, there is a lot of complex optimization math that goes into building this from scratch. Tune provides a scalable and easy-to-use implementation of the SOTA PBT algorithm




In [71]:
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

def get_scheduler():
    #Creating the PBT scheduler
    scheduler = PopulationBasedTraining(
        mode = "max",
        metric='eval_acc',
        #perturbation_interval=2,
        hyperparam_mutations={
            "weight_decay": tune.choice([0.0, 0.3]),
            "learning_rate": tune.choice([1e-5, 5e-5]),
            "per_device_train_batch_size": tune.choice([16, 64]),
            "num_train_epochs": tune.choice([2,5]),
            "warmup_steps": tune.choice(range(0, 500))
        }
    )
    return scheduler

We run only 8 trials, much less than Bayesian Optimization since instead of stopping bad trials, they copy from the good ones.

In [60]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb",
    #hp_space=wandb_hp_space,
    n_trials=8,
    #keep_checkpoints_num=1,
    scheduler=get_scheduler()
)

Create sweep with ID: ile8sxcl
Sweep URL: https://wandb.ai/anudeepvanjavakam/reddit_product_tagging/sweeps/ile8sxcl


wandb: Agent Starting Run: zggjdxl0 with config:
wandb: 	learning_rate: 6.187091203956564e-05
wandb: 	num_train_epochs: 4
wandb: 	per_device_train_batch_size: 64
wandb: 	seed: 8
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to b

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'seed' was locked by 'sweep' (ignored update).


...

Trying to set _wandb in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set assignments in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Trying to set metric in the hyperparameter search but there is no corresponding field in `TrainingArguments`.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to b

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'seed' was locked by 'sweep' (ignored update).


{'loss': 0.2209, 'learning_rate': 6.271039073775904e-05, 'epoch': 0.45}
{'eval_loss': 0.2422882318496704, 'eval_precision': 0.5013333333333333, 'eval_recall': 0.3484708063021316, 'eval_f1': 0.41115363586659376, 'eval_accuracy': 0.9451498439570775, 'eval_runtime': 61.197, 'eval_samples_per_second': 21.03, 'eval_steps_per_second': 2.631, 'epoch': 0.45}
{'loss': 0.1492, 'learning_rate': 5.152409199752375e-05, 'epoch': 0.91}
{'eval_loss': 0.3045324981212616, 'eval_precision': 0.6195121951219512, 'eval_recall': 0.3531047265987025, 'eval_f1': 0.4498229043683589, 'eval_accuracy': 0.9448505835577786, 'eval_runtime': 71.46, 'eval_samples_per_second': 18.01, 'eval_steps_per_second': 2.253, 'epoch': 0.91}
{'loss': 0.0875, 'learning_rate': 4.033779325728846e-05, 'epoch': 1.36}
{'eval_loss': 0.32817161083221436, 'eval_precision': 0.6302816901408451, 'eval_recall': 0.33178869323447635, 'eval_f1': 0.4347298117789921, 'eval_accuracy': 0.9458766192125176, 'eval_runtime': 61.267, 'eval_samples_per_secon

eval/accuracy,▂▁▅▄▅█
eval/f1,▁█▅▅▇█
eval/loss,▁▅▆▄▇█
eval/precision,▁▇█▃▄▇
eval/recall,▄▅▁▆█▆
eval/runtime,▁█▁▃▃▃
eval/samples_per_second,█▁█▆▆▆
eval/steps_per_second,█▁█▆▆▆
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇█
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇█
train/learning_rate,█▇▅▄▂▁


In [67]:
best_trial

BestRun(run_id='kgy00di9', objective=2.423856367441647, hyperparameters={'learning_rate': 5.6309788887288427e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 8, 'seed': 25, 'assignments': {}, 'metric': 'eval/loss'}, run_summary=None)

In [74]:
for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


{'loss': 0.185, 'learning_rate': 4.35353358729308e-05, 'epoch': 0.91}
{'eval_loss': 0.24270012974739075, 'eval_precision': 0.5070603337612324, 'eval_recall': 0.366079703429101, 'eval_f1': 0.4251883745963402, 'eval_accuracy': 0.9448933350433928, 'eval_runtime': 80.7282, 'eval_samples_per_second': 15.942, 'eval_steps_per_second': 1.994, 'epoch': 0.91}
{'loss': 0.0794, 'learning_rate': 3.0760882858573164e-05, 'epoch': 1.81}
{'eval_loss': 0.2586621344089508, 'eval_precision': 0.6069868995633187, 'eval_recall': 0.386468952734013, 'eval_f1': 0.47225368063420153, 'eval_accuracy': 0.9482707024069087, 'eval_runtime': 61.545, 'eval_samples_per_second': 20.912, 'eval_steps_per_second': 2.616, 'epoch': 1.81}
{'loss': 0.0404, 'learning_rate': 1.798642984421554e-05, 'epoch': 2.72}
{'eval_loss': 0.28830501437187195, 'eval_precision': 0.577023498694517, 'eval_recall': 0.40963855421686746, 'eval_f1': 0.47913279132791325, 'eval_accuracy': 0.9482279509212945, 'eval_runtime': 60.501, 'eval_samples_per_sec

TrainOutput(global_step=2204, training_loss=0.07536158587668638, metrics={'train_runtime': 3274.7055, 'train_samples_per_second': 5.378, 'train_steps_per_second': 0.673, 'train_loss': 0.07536158587668638, 'epoch': 4.0})

In [77]:
# after pushing the best model to hugging face hub (kgy00di9)
model = AutoModelForTokenClassification.from_pretrained("anudeepvanjavakam/distilbert_finetuned_wnut17_wandb_ner")
trainer = Trainer(model=model)
tokenizer = AutoTokenizer.from_pretrained("anudeepvanjavakam/distilbert_finetuned_wnut17_wandb_ner")

C:\Users\avanjavakam\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\avanjavakam\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [80]:
text="""Apple unveils all-new MacBook Air, supercharged by the new M2 chip"""

print(tag_sentence(text))

         word            tag
0       [CLS]              O
1       apple  B-corporation
2          un              O
3        ##ve              O
4       ##ils              O
5         all              O
6           -              O
7         new              O
8         mac      B-product
9      ##book      I-product
10        air      I-product
11          ,              O
12      super              O
13  ##charged              O
14         by              O
15        the              O
16        new              O
17         m2      B-product
18       chip      I-product
19      [SEP]              O


## Testing other options for hypertuning (without population based training)

In [63]:
# method
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 1
        },
    'batch_size': {
        'values': [8, 64]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 5e-5
    },
    'weight_decay': {
        'values': [0.0, 0.3]
    },
}


sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project='reddit_product_tagging')

Create sweep with ID: wx2o1mj0
Sweep URL: https://wandb.ai/anudeepvanjavakam/reddit_product_tagging/sweeps/wx2o1mj0


In [64]:
def train(config=None):
  with wandb.init(config=config):
    # set sweep configuration
    config = wandb.config


    # set training arguments
    training_args = TrainingArguments(
        output_dir='reddit_product_tagging-sweeps',
	      report_to='wandb',  # Turn on Weights & Biases logging
        num_train_epochs=config.epochs,
        learning_rate=config.learning_rate,
        weight_decay=config.weight_decay,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=16,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,
        disable_tqdm=False
    )

    # define training loop
    trainer = Trainer(
        # model,
        model_init=model_init,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_wnut['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )


    # start training loop
    trainer.train()

In [65]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 70nqsf42 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 1
wandb: 	learning_rate: 3.0218375436134713e-05
wandb: 	weight_decay: 0.3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

  0%|          | 0/551 [00:00<?, ?it/s]

{'loss': 0.2136, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/81 [00:00<?, ?it/s]

...

{'loss': 0.7945, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/81 [00:00<?, ?it/s]

c:\Users\avanjavakam\.conda\envs\lit_or_not_on_reddit\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\avanjavakam\.conda\envs\lit_or_not_on_reddit\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.44381245970726013, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9256124150314223, 'eval_runtime': 80.581, 'eval_samples_per_second': 15.972, 'eval_steps_per_second': 1.005, 'epoch': 1.0}
{'train_runtime': 617.4248, 'train_samples_per_second': 7.131, 'train_steps_per_second': 0.112, 'train_loss': 0.7944799229718637, 'epoch': 1.0}


eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁


In [66]:
wandb.finish()

In [81]:
## Results and Comparison Table

comparison_table = pd.read_csv("comparison_table_hyperparameter_tuning/wandb_export_2023-06-05T21_34_26.822-07_00.csv")
comparison_table

,Name,eval/accuracy,eval/f1,eval/loss,eval/precision,eval/recall,batch_size,epochs,eval_batch_size,eval_steps,evaluation_strategy,learning_rate,per_device_eval_batch_size,per_device_train_batch_size,Runtime,Sweep,train_batch_size
0,floral-sweep-3 (chosen model),95%,48%,32%,58%,41%,NaN,NaN,8,500.0,steps,0.000056,8,8,3036,ile8sxcl,8
1,zesty-sweep-6,95%,48%,39%,56%,42%,NaN,NaN,8,500.0,steps,0.000037,8,4,6402,ile8sxcl,4
2,colorful-sweep-2,95%,47%,33%,60%,39%,NaN,NaN,8,500.0,steps,0.000067,8,16,3041,ile8sxcl,16
3,gentle-sweep-1,95%,48%,26%,54%,43%,NaN,NaN,8,500.0,steps,0.000062,8,64,2155,ile8sxcl,64
4,kind-sweep-5,95%,47%,26%,58%,40%,NaN,NaN,8,500.0,steps,0.000047,8,32,1667,ile8sxcl,32
5,pleasant-sweep-8,95%,45%,35%,60%,36%,NaN,NaN,8,500.0,steps,0.000074,8,4,3166,ile8sxcl,4
6,celestial-sweep-7,95%,45%,25%,54%,38%,NaN,NaN,8,500.0,steps,0.000067,8,64,1214,ile8sxcl,64
7,drawn-sweep-9,95%,44%,24%,56%,36%,8.0,1.0,16,NaN,epoch,0.000047,16,8,835,wx2o1mj0,8
8,pleasant-sweep-1,94%,44%,25%,57%,35%,8.0,1.0,16,NaN,epoch,0.000030,16,8,855,wx2o1mj0,8
9,crimson-sweep-2,94%,30%,29%,44%,23%,8.0,1.0,16,NaN,epoch,0.000011,16,8,834,wx2o1mj0,8


## Results

A low F1 score but high accuracy is observed. when it comes to the F1 score, which is a measure of the balance between precision and recall, DistilBERT may sometimes show slightly lower performance compared to the larger BERT model or other more complex models. This is because DistilBERT sacrifices some model capacity to achieve faster inference times and a smaller model size. Consequently, it may have slightly lower precision or recall compared to larger models, leading to a lower F1 score in some cases.

Class imbalance: The dataset is imbalanced, meaning one class has significantly more samples than the other, the model might have a high accuracy by predicting the majority class correctly most of the time. However, it may struggle with the minority class, resulting in lower recall and F1 score for that class.



## Conclusion

- Recap of the this model objective: Find product mentions amongst posts and comments via named entity recognition (NER). This model can be used by the app to help users with queries such as "Best 4K TV to buy"

- Methodology: a baseline model is developed for initial evaluation. A pre-trained DistilBERT model (token-classification) from Hugging Face Transformers which did not have product entities is fine tuned with the readily available wnut_17 data set to predict products. Hyperparameter tuning with "Weights and Biases" (provides tools to quickly track experiments, version and iterate on datasets, evaluate model performance, reproduce models, and visualize results) was performed to find the best model with an accuracy of 95% evaluated and tested on examples. This best model is pushed to the Hugging Face hub and can be called for inference through the app.




## Improvement

Examine the data and check how many examples have each tags/token. Model needs more product data to be trained on.

Cross-validation (with a stratified split since classes aren’t balanced) may help mitigate some of the problems that come from doing a train/test split with small datasets.

Class imbalance: there is a class imbalance (model may predict the majority class “O” majority of the time) and needs to be addressed by employing techniques like oversampling the minority class, undersampling the majority class, or using class weighting.

Consider other evaluation metrics.

Fine-tuning with more reddit comments and product entities can help the model learn better representations and improve its performance on the task at hand.

## Acknowledgement:
- https://huggingface.co/datasets/wnut_17
- https://huggingface.co/learn/nlp-course/chapter7/2
- https://towardsdatascience.com/named-entity-recognition-with-deep-learning-bert-the-essential-guide-274c6965e2d
- https://towardsdatascience.com/build-a-named-entity-recognition-app-with-streamlit-f157672f867f
- https://github.com/JINHXu/create-annotated-NER-dataset
- https://wandb.ai/matt24/vit-snacks-sweeps/reports/Hyperparameter-Search-with-W-B-Sweeps-for-Hugging-Face-Transformer-Models--VmlldzoyMTUxNTg0
- https://docs.wandb.ai/guides/integrations/huggingface#2-name-the-project
- https://docs.wandb.ai/guides/sweeps/initialize-sweeps
- https://wandb.ai/amogkam/transformers/reports/Hyperparameter-Optimization-for-Huggingface-Transformers--VmlldzoyMTc2ODI
- https://huggingface.co/docs/transformers/hpo_train
- https://docs.streamlit.io/library/advanced-features/caching
